In [8]:
import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import numpy as np
import argparse
import time
import logging
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
from config import (sample_rate, classes_num, mel_bins, fmin, fmax, window_size, 
    hop_size, window, pad_mode, center, ref, amin, top_db)
from losses import get_loss_func
from pytorch_utils import move_data_to_device, do_mixup
from utilities import (create_folder, get_filename, create_logging, StatisticsContainer, Mixup)
from data_generator import GtzanDataset, TrainSampler, EvaluateSampler, collate_fn
from models import Transfer_Cnn14
from evaluate import Evaluator

In [30]:
DATASET_DIR="D:\\Py_things\\PyProjects\\Kvant_projects\\PANNS_R_TEST\\training\\datasets\\R_SOUNDS"
WORKSPACE="D:\\Py_things\\PyProjects\\Kvant_projects\\PANNS_R_TEST\\training\\workspaces\\music"
PRETRAINED_CHECKPOINT_PATH="D:\\Py_things\\PyProjects\\Kvant_projects\\PANNS_R_TEST\\panns_data\\Cnn14_mAP=0.431.pth"
FILENAME ='D:\\Py_things\\PyProjects\\Kvant_projects\\PANNS_R_TEST\\TrainPANNS\\fine-tuning-music-example\\jup_test\\run_main_jup'

In [34]:
def train():

    # Arugments & parameters
#     dataset_dir = args.dataset_dir
#     workspace = args.workspace
#     holdout_fold = args.holdout_fold
#     model_type = args.model_type
#     pretrained_checkpoint_path = args.pretrained_checkpoint_path
#     freeze_base = args.freeze_base
#     loss_type = args.loss_type
#     augmentation = args.augmentation
#     learning_rate = args.learning_rate
#     batch_size = args.batch_size
#     resume_iteration = args.resume_iteration
#     stop_iteration = args.stop_iteration
#     device = 'cuda' if (args.cuda and torch.cuda.is_available()) else 'cpu'
#     filename = args.filename
#     num_workers = 8

# впишем ручками параметры для тренировки
    dataset_dir = DATASET_DIR
    workspace = WORKSPACE
    holdout_fold = 1
    model_type = "Transfer_Cnn14"
    pretrained_checkpoint_path = PRETRAINED_CHECKPOINT_PATH
    freeze_base = False # это что?, пока так
    loss_type = "clip_nll" #добавил ковычек
    augmentation = 'mixup'
    learning_rate = 1e-4
    batch_size = 8
    resume_iteration = 0
    stop_iteration = 10 #пока чуть-чуть, но нужно сильно больше
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    filename = FILENAME #Пока так
    num_workers = 8

    loss_func = get_loss_func(loss_type)
    pretrain = True if pretrained_checkpoint_path else False
    
    hdf5_path = os.path.join(workspace, 'features', 'waveform.h5')

    checkpoints_dir = os.path.join(workspace, 'checkpoints', filename, 
        'holdout_fold={}'.format(holdout_fold), model_type, 'pretrain={}'.format(pretrain), 
        'loss_type={}'.format(loss_type), 'augmentation={}'.format(augmentation),
         'batch_size={}'.format(batch_size), 'freeze_base={}'.format(freeze_base))
    create_folder(checkpoints_dir)

    statistics_path = os.path.join(workspace, 'statistics', filename, 
        'holdout_fold={}'.format(holdout_fold), model_type, 'pretrain={}'.format(pretrain), 
        'loss_type={}'.format(loss_type), 'augmentation={}'.format(augmentation), 
        'batch_size={}'.format(batch_size), 'freeze_base={}'.format(freeze_base), 
        'statistics.pickle')
    create_folder(os.path.dirname(statistics_path))
    
    logs_dir = os.path.join(workspace, 'logs', filename, 
        'holdout_fold={}'.format(holdout_fold), model_type, 'pretrain={}'.format(pretrain), 
        'loss_type={}'.format(loss_type), 'augmentation={}'.format(augmentation), 
        'batch_size={}'.format(batch_size), 'freeze_base={}'.format(freeze_base))
    create_logging(logs_dir, 'w')
#     logging.info(args) # Пока закомитить

    if 'cuda' in device:
        logging.info('Using GPU.')
    else:
        logging.info('Using CPU. Set --cuda flag to use GPU.')

    print('window size {}'.format(window_size))

    # Model
    Model = eval(model_type)
    model = Model(sample_rate, window_size, hop_size, mel_bins, fmin, fmax, 
        classes_num, freeze_base)

    # Statistics
    statistics_container = StatisticsContainer(statistics_path)

    if pretrain:
        logging.info('Load pretrained model from {}'.format(pretrained_checkpoint_path))
        model.load_from_pretrain(pretrained_checkpoint_path)

    if resume_iteration:
        resume_checkpoint_path = os.path.join(checkpoints_dir, '{}_iterations.pth'.format(resume_iteration))
        logging.info('Load resume model from {}'.format(resume_checkpoint_path))
        resume_checkpoint = torch.load(resume_checkpoint_path)
        model.load_state_dict(resume_checkpoint['model'])
        statistics_container.load_state_dict(resume_iteration)
        iteration = resume_checkpoint['iteration']
    else:
        iteration = 0

    # Parallel
    print('GPU number: {}'.format(torch.cuda.device_count()))
    model = torch.nn.DataParallel(model)

    dataset = GtzanDataset()

    # Data generator
    train_sampler = TrainSampler(
        hdf5_path=hdf5_path, 
        holdout_fold=holdout_fold, 
        batch_size=batch_size * 2 if 'mixup' in augmentation else batch_size)

    validate_sampler = EvaluateSampler(
        hdf5_path=hdf5_path, 
        holdout_fold=holdout_fold, 
        batch_size=batch_size)

    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=dataset, 
        batch_sampler=train_sampler, collate_fn=collate_fn, 
        num_workers=num_workers, pin_memory=True)

    validate_loader = torch.utils.data.DataLoader(dataset=dataset, 
        batch_sampler=validate_sampler, collate_fn=collate_fn, 
        num_workers=num_workers, pin_memory=True)

    if 'cuda' in device:
        model.to(device)

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999),
        eps=1e-08, weight_decay=0., amsgrad=True)

    
    if 'mixup' in augmentation:
        mixup_augmenter = Mixup(mixup_alpha=1.)
     
    # Evaluator
    evaluator = Evaluator(model=model)
    
    train_bgn_time = time.time()
    
    # Train on mini batches
    for batch_data_dict in train_loader:

        # import crash
        # asdf
        
        # Evaluate
        if iteration % 200 == 0 and iteration > 0:
            if resume_iteration > 0 and iteration == resume_iteration:
                pass
            else:
                logging.info('------------------------------------')
                logging.info('Iteration: {}'.format(iteration))

                train_fin_time = time.time()

                statistics = evaluator.evaluate(validate_loader)
                logging.info('Validate accuracy: {:.3f}'.format(statistics['accuracy']))

                statistics_container.append(iteration, statistics, 'validate')
                statistics_container.dump()

                train_time = train_fin_time - train_bgn_time
                validate_time = time.time() - train_fin_time

                logging.info(
                    'Train time: {:.3f} s, validate time: {:.3f} s'
                    ''.format(train_time, validate_time))

                train_bgn_time = time.time()

        # Save model 
        if iteration % 2000 == 0 and iteration > 0:
            checkpoint = {
                'iteration': iteration, 
                'model': model.module.state_dict()}

            checkpoint_path = os.path.join(
                checkpoints_dir, '{}_iterations.pth'.format(iteration))
                
            torch.save(checkpoint, checkpoint_path)
            logging.info('Model saved to {}'.format(checkpoint_path))
        
        if 'mixup' in augmentation:
            batch_data_dict['mixup_lambda'] = mixup_augmenter.get_lambda(len(batch_data_dict['waveform']))
        
        # Move data to GPU
        for key in batch_data_dict.keys():
            batch_data_dict[key] = move_data_to_device(batch_data_dict[key], device)
        
        # Train
        model.train()

        if 'mixup' in augmentation:
            batch_output_dict = model(batch_data_dict['waveform'], 
                batch_data_dict['mixup_lambda'])
            """{'clipwise_output': (batch_size, classes_num), ...}"""

            batch_target_dict = {'target': do_mixup(batch_data_dict['target'], 
                batch_data_dict['mixup_lambda'])}
            """{'target': (batch_size, classes_num)}"""
        else:
            batch_output_dict = model(batch_data_dict['waveform'], None)
            """{'clipwise_output': (batch_size, classes_num), ...}"""

            batch_target_dict = {'target': batch_data_dict['target']}
            """{'target': (batch_size, classes_num)}"""

        # loss
        loss = loss_func(batch_output_dict, batch_target_dict)
        print(iteration, loss)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Stop learning
        if iteration == stop_iteration:
            break 

        iteration += 1

In [35]:
# --dataset_dir=$DATASET_DIR 
# --workspace=$WORKSPACE 
# --holdout_fold=1 
# --model_type="Transfer_Cnn14" 
# --pretrained_checkpoint_path=$PRETRAINED_CHECKPOINT_PATH 
# --loss_type=clip_nll 
# --augmentation='mixup' 
# --learning_rate=1e-4 
# --batch_size=8 
# --resume_iteration=0 
# --stop_iteration=10 
# --cuda

In [36]:
train()

root        : INFO     Using CPU. Set --cuda flag to use GPU.
root        : INFO     Using CPU. Set --cuda flag to use GPU.


window size 1024
window size 1024


root        : INFO     Load pretrained model from D:\Py_things\PyProjects\Kvant_projects\PANNS_R_TEST\panns_data\Cnn14_mAP=0.431.pth
root        : INFO     Load pretrained model from D:\Py_things\PyProjects\Kvant_projects\PANNS_R_TEST\panns_data\Cnn14_mAP=0.431.pth


GPU number: 0


transformers.file_utils: INFO     PyTorch version 2.1.0 available.
transformers.file_utils: INFO     PyTorch version 2.1.0 available.
transformers.modeling_xlnet: INFO     Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
transformers.modeling_xlnet: INFO     Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


0 tensor(0.2017, grad_fn=<NegBackward0>)
1 tensor(0.0795, grad_fn=<NegBackward0>)
2 tensor(0.1132, grad_fn=<NegBackward0>)
3 tensor(0.1167, grad_fn=<NegBackward0>)
4 tensor(0.1649, grad_fn=<NegBackward0>)
5 tensor(0.1435, grad_fn=<NegBackward0>)
6 tensor(0.0927, grad_fn=<NegBackward0>)
7 tensor(0.4546, grad_fn=<NegBackward0>)
8 tensor(0.1858, grad_fn=<NegBackward0>)
9 tensor(0.0934, grad_fn=<NegBackward0>)
10 tensor(0.1245, grad_fn=<NegBackward0>)


In [29]:
FILENAME

'D:\\Py_things\\PyProjects\\Kvant_projects\\PANNS_R_TEST\\TrainPANNS\x0cine-tuning-music-example\\jup_test\run_main_jup'